<a href="https://colab.research.google.com/github/jfdoppler/DNL_1c2021/blob/main/08_modos_empiricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modos Empíricos con SVD

En este Notebook vamos a trabajar con datos experimentales correspondientes a una película del aparato fonador de un ave. En este sistema una membrana oscila debedio a la interacción con un flujo de aire (mismo fenómeno que sucede, por ejemplo, en nuestras cuerdas vocales). 
El Notebook está basado en la versión del curso de sistemas dinámicos dictado por Gabriel Mindlin y Gonzalo Uribarri en 2020. Gracias por el material. 


Aquí la película en cuestión:
https://www.pnas.org/content/suppl/2020/02/12/1922147117.DCSupplemental

### 1. Carpeta de Trabajo y Lectura de Archivos

Primero, vamos a montarnos sobre el directorio de google Drive. Esto nos permitira leer y guardar archivos en nuestro Drive. Generamos la dirección `root_dir` (un string) que apunta a la carpeta de trabajo. Noten que al ejecutar el comando `mount`, google nos pedirá una contraseña la cual obtendremos mediante el link que aparece.

**Nota:** Recuerden que primero deben crear en su drive la carpeta.

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

# Aca deben apuntar a la carpeta de su drive donde guardaron el gif
# https://drive.google.com/file/d/1Nekjm2Xg4RL1ZwUXkeH07NnlGfvthnDG/view?usp=sharing
root_dir = "/content/gdrive/My Drive/Ayudantia/DNL/ModosEmpiricos/pelicula_original.gif"

Para importar separar el gif en frames definimos la función importar_gif

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from numpy import asarray
from mpl_toolkits.mplot3d import Axes3D

def importar_gif(dir_gif):
  im = Image.open(dir_gif)
  lista=[asarray(im)]
  # To iterate through the entire gif
  try:
      while 1:
          im.seek(im.tell()+1)
          lista.append(asarray(im))
          
  except EOFError:
      pass # end of sequence

  frames=np.array(lista)
  frames=-frames.astype('float32')+255. 
  return frames

In [ ]:
x = importar_gif(root_dir)
print(x.shape)

Entonces tenemos 127 frames, de 160 píxeles de altura por 150 píxeles de base. Veamos alguno de estos frames.

In [ ]:
plt.imshow(x[10],cmap='gray')

### 2. Preprocesamiento

Primero vamos a adecuar las imágenes con las que vamos a trabajar, es decir vamos a llevarlas a una forma adecauda para poder aplicarles el algortimo de SVD.

In [ ]:
# X es un tensor de mxhxb con n = # de frames, h=altura, b=base.

# Normalizamos: llevamos de [0,255] a [0,1]
X = x/255

# Restamos la imagen Media
X_medio = np.mean(X,axis=0)
X = X-X_medio

In [ ]:
# Graficamos el frame promedio
plt.imshow(X_medio,cmap='gray')
plt.colorbar()
plt.show()

In [ ]:

# Graficamos un frame renormalizado como ejemplo
numero = 55
plt.imshow(X[numero],cmap='gray')
plt.colorbar()
plt.show()

Por ultimo, vamos "aplastar" el tensor que contiene las imágenes. Es decir que vamos a llevarlo a una forma $(n,m)$ siendo $n = altura \times base$ y siendo $ m = \# \  frames$ .

In [ ]:
# Tomamos las dimensiones de X
dims = np.shape(X)

# Construyo matriz Y de nxm con n = altura x base, m = num de frames
Y = np.transpose(np.reshape(X,(dims[0],dims[1]*dims[2])))

print('Shape de la matrix original',np.shape(X))
print('Shape de la matrix aplanada',np.shape(Y))


### 3. Aplicamos SVD

Usaremos la función de la libreria de algebra lineal de numpy `np.linalg.svd` para realizar la descomposición.

In [ ]:
#SVD para los primeros N frames
N = 127

# # SVD Completo
# Uhat, Shat, Vhat = np.linalg.svd(Y[:,:N],full_matrices=True)

# SVD Económico
Uhat, Shat, Vhat = np.linalg.svd(Y[:,:N], full_matrices=False)

print('Shape de U:',Uhat.shape)
print('Shape de Shat:',Shat.shape)
print('Shape de Vhat:',Vhat.shape)

**Ejercicio:** 
Cambie del SVD económico al SVD completo y vea como cambia la forma de las matrices. ¿Qué es lo que está cambiando entre ambos manera de computar la descomposición?

Vamos a graficar el valor de cada uno de los valores singulares y la suma cumulativa de los mismos:

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(Shat,'o')
plt.ylabel('Singular value')
plt.grid()
plt.yscale('linear')
plt.subplot(1,2,2)
plt.plot(np.cumsum(Shat)/np.sum(Shat),'o')
plt.ylabel('Suma cumulativa')
plt.grid()


Noten que en la matriz Uhat guardamos la información de los modos espaciales como columnas, mientras que en la matriz Vhat guardamos la evolucion temporal de cada uno de estos modos como filas.

Veamos como lucen los primeros 6 modos espaciales. 

In [ ]:
plt.figure(figsize=(22,10))
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.imshow(np.reshape(Uhat[:,i],(dims[1],dims[2])),cmap='gray',vmin=np.min(Uhat[:,:6]),vmax=np.max(Uhat[:,:6]))
  plt.xticks([])
  plt.yticks([])
  plt.colorbar()

#### 3. Evolucion Temporal de los modos

Exploremos como evolucionan temporalmente los primeros 4 modos empíricos.

In [ ]:
#PRIMEROS MODOS TEMPORALES

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(Vhat[0],'.-',label='V1')
plt.plot(Vhat[1],'.-',label='V2')
plt.plot(Vhat[2],'.-',label='V3')
#plt.plot(Vhat[3],'.-',label='V3')
plt.title('Evolución temporal del los primeros 4 modos')
plt.legend()
plt.subplot(1,2,2)

modo = 0
plt.plot(Vhat[modo],'.-',label='Modo elegido')
plt.title('Evolución temporal del un modo')
plt.legend()


plt.show()

Si graficamos la evolución temporal de los primeros 3 modos:

In [ ]:
fig = plt.figure(figsize=(15,6))
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax = fig.gca(projection='3d')
ax.plot(Vhat[0],Vhat[1],Vhat[2])
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()

**Discusión:** Observando la imagen de los modos espaciales y su evolución temporal, responda:



*   ¿Podríamos pensar que la evoilcuión de los priemros 3 modos se da dentro de un espacio de fases? ¿Que tipo de solución sería?

*   Si tuviese que elegir dos modos que representen la dinamica de oscilación, ¿cuáles eligiríá? Haga un grafico con únicamente esos modos.

*   Observe cómo hubiesen quedado los primeros modos si no se restaba el "frame promedio". Mire el priemr modo espacial en ese caso ¿Parece estar involucrado en la dinámica de la membrana que oscila? ¿A que podría corresponder?

 


#### 4. Reconstrucción de la pelicula a partir de los modos truncados

Por último, vamos a reconstruir la película, pero únicamente utilizando una cantidad truncada de modos.

In [ ]:
modes=[0,1,2,3,4,5,6] #qué modos quiero usar para reconstrucción
Y_approx=np.dot(np.dot(Uhat[:,modes],np.diag(Shat[modes])),Vhat[modes,:])

lista=[]
for j in range(X.shape[0]):
  #Reacomodo las dimensiones para recuoerar la forma original de los frames
  N=np.reshape(Y_approx[:,j],(dims[1],dims[2]))+X_medio
  #Reescaleo de 0 a 255
  N=N*255 
  N=np.clip(N, 0, 255)
  N=N.astype('uint8')
  lista.append(N)
  

Guardamos el nuevo gif reconstruido en nuestra carpeta de trabajo:

In [ ]:
import imageio
from IPython.display import Image
dir_gif = "/content/gdrive/My Drive/Ayudantia/DNL/ModosEmpiricos/proyeccion.gif"
imageio.mimsave(dir_gif, lista)

Podemos observar como quedó nuestra pelicula realizada unicamente con N modos.

In [ ]:
print('Pelicula generada con 7 modos')
Image(open(dir_gif,'rb').read())

**Ejercicio:** Cambie el número de componentes utilizadas y observe como cambia la calidad de la animación reconstruida.

# Usar SVD para comprimir un foto

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import cv2 as cv
from tensorflow.keras.preprocessing import image
#Guardan la imagen en su directorio
#https://drive.google.com/file/d/1Q11MUlfuKHK7jo2xQHimkUZ1J71VlDiM/view?usp=sharing
# Leemos la imagen de internet
#image = io.imread('https://drive.google.com/file/d/1Q11MUlfuKHK7jo2xQHimkUZ1J71VlDiM/view?usp=sharing')
dir_img="/content/gdrive/My Drive/Ayudantia/DNL/ModosEmpiricos/messiFHD.jpg"
im = image.load_img(dir_img, color_mode="grayscale")
img=image.img_to_array(im)
X_img=img[:,:,0]
print('Dimension de la foto:')
print(X_img.shape)

Tomamos como matriz la foto en escala de grises, y hacemos la descompocisión SVD.

In [ ]:
U, D, V = np.linalg.svd(X_img)

Graficamos como queda la foto reconstruida únicamente con N modos y descubrimos la imagen:

In [ ]:
for i in [2, 3, 5, 8, 10, 20, 50, 100]:
    reconstimg = np.matrix(U[:, :i]) * np.diag(D[:i]) * np.matrix(V[:i, :])
    plt.figure(figsize=(8,8))
    plt.imshow(reconstimg, cmap='gray')
    title = "N = %s" % i
    plt.title(title)
    plt.show()

Guardemos la reconstrucción con 100 modos y veamos cuanto pesa en comparación a la original.

In [ ]:
reconstimg=np.array(reconstimg)

In [ ]:
import matplotlib

matplotlib.image.imsave('/content/gdrive/My Drive/Ayudantia/DNL/ModosEmpiricos/outfile.jpg', reconstimg,cmap='Greys_r')